In [1]:
# !pip install GML

In [2]:
# !pip install autofeat

In [3]:
# !pip install sweetviz

In [4]:
# !pip install heamy

In [5]:
# !pip install optuna

In [6]:
import pandas as pd

import numpy as np

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, PolynomialFeatures, RobustScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.neural_network import MLPRegressor
from sklearn.cluster import KMeans
from sklearn.ensemble import ExtraTreesRegressor, VotingRegressor, HistGradientBoostingRegressor, BaggingRegressor
from sklearn.linear_model import BayesianRidge, LinearRegression, LassoLars, Ridge, ElasticNet, Lasso
from sklearn.inspection import permutation_importance

from mlxtend.regressor import StackingCVRegressor

# import sweetviz as sv

# import smogn

# from heamy.dataset import Dataset
# from heamy.estimator import Regressor, Classifier
# from heamy.pipeline import ModelsPipeline

import matplotlib.pyplot as plt

import seaborn as sns

from scipy import stats

from xgboost import XGBRegressor

from catboost import CatBoostRegressor

from lightgbm import LGBMRegressor

import tqdm

from category_encoders import TargetEncoder

from scipy.stats import zscore

# import optuna

# from autofeat import AutoFeatRegressor

# from pystacknet.pystacknet import StackNetRegressor

# from GML.Ghalat_Machine_Learning import Ghalat_Machine_Learning

In [7]:
def accuracy(y_true,y_pred):
  return 100 * max(0,1-np.sqrt(mean_squared_error(y_true,y_pred)))

In [8]:
path = '/kaggle/input/he-mlcomp/'

In [9]:
train = pd.read_csv(path+'Train.csv')
test = pd.read_csv(path+'Test.csv')

In [10]:
"""
test = train[5000:].copy()
test_labels = test['Attrition_rate']
test.drop(['Attrition_rate'],axis=1,inplace=True)
train = train[:5000].copy()
"""
pass

In [11]:
test_eids = test['Employee_ID'].copy()

In [12]:
all_data = pd.concat([train,test],axis=0)

In [13]:
all_data['Salary_Now'] = all_data['Pay_Scale'] * 10000
all_data['Salary_In_Next_Year'] = all_data['Salary_Now'] + all_data['Salary_Now'] * (all_data['growth_rate']/100)
all_data['Salary_In_Next_10_Years'] = all_data['Salary_Now'] + all_data['Salary_Now'] * (all_data['growth_rate']/100)*10

In [14]:
all_data['Perc_Life_Spent'] = 100 - ((all_data['Age'] - all_data['Time_of_service'])/all_data['Age']) * 100

In [15]:
def pos_tos_tsp(s):
    return str(s['Post_Level']) + '_' + str(s['Time_of_service']) + '_' + str(s['Time_since_promotion'])

all_data['pos_tos_tsp'] = all_data.apply(pos_tos_tsp,axis=1)

In [16]:
all_data['Sat_Level'] = all_data['Post_Level'] + all_data['Time_of_service'] + all_data['Time_since_promotion']

In [17]:
def avg_g_tos(s):
    gr = s['growth_rate']
    tos = s['Time_of_service']
    res = train[(train['growth_rate'] <= (gr + 4)) & (train['growth_rate'] >= (gr - 4)) & 
          (train['Time_of_service'] <= (tos + 4)) & (train['Time_of_service'] >= (tos - 4))].loc[:,'Attrition_rate'].describe()
    if res['mean'] != np.nan:
        return res['mean']
    else:
        return train['Attrition_rate'].mean()

In [18]:
all_data['avg_g_tos'] = all_data.apply(avg_g_tos,axis=1)

In [19]:
def UxH(x):
    return str(x['Unit']) + '_' + str(x['Hometown'])

def XxBxH(x):
    return str(x['Compensation_and_Benefits']) + '_' + str(x['Hometown'])

def XxBxU(x):
    return str(x['Compensation_and_Benefits']) + '_' + str(x['Unit'])

In [20]:
def emp_Type(s):
    if s['Time_of_service'] == 0.0:
        return "New"
    elif s['Time_of_service'] <= 5.0:
        return "Moderate"
    elif s['Time_of_service'] <= 10.0:
        return "Senior"
    else:
        return "SuperSenior"

In [21]:
def person_Type(s):
    if s['Age'] <= 20:
        return 'Teenager'
    elif s['Age'] <= 40:
        return "Man"
    elif s['Age'] <= 60:
        return "Uncle"
    else:
        return "AboutToDie"

In [22]:
all_data['Emp_Type'] = all_data.apply(emp_Type,axis=1)

all_data['person_Type'] = all_data.apply(person_Type,axis=1)

all_data['UxH'] = all_data.apply(UxH,axis=1)

In [23]:
def retired(s):
    if s['Age'] >= 45:
        return 1
    return 0

In [24]:
all_data['Retiring_or_not'] = all_data.apply(retired,axis=1)

In [25]:
all_data.columns

Index(['Employee_ID', 'Gender', 'Age', 'Education_Level',
       'Relationship_Status', 'Hometown', 'Unit', 'Decision_skill_possess',
       'Time_of_service', 'Time_since_promotion', 'growth_rate', 'Travel_Rate',
       'Post_Level', 'Pay_Scale', 'Compensation_and_Benefits',
       'Work_Life_balance', 'VAR1', 'VAR2', 'VAR3', 'VAR4', 'VAR5', 'VAR6',
       'VAR7', 'Attrition_rate', 'Salary_Now', 'Salary_In_Next_Year',
       'Salary_In_Next_10_Years', 'Perc_Life_Spent', 'pos_tos_tsp',
       'Sat_Level', 'avg_g_tos', 'Emp_Type', 'person_Type', 'UxH',
       'Retiring_or_not'],
      dtype='object')

In [26]:
all_data['CMP_BF_HT'] = all_data.apply(XxBxH,axis=1)
all_data['CMP_BF_U'] = all_data.apply(XxBxU,axis=1)

In [27]:
all_data['Perc_Spent_WO_Promotion'] = all_data['Time_since_promotion'] / all_data['Time_of_service']

In [28]:
all_data['First_Prom'] = all_data['Time_of_service'] - all_data['Time_since_promotion']

In [29]:
# train['zscore'] = zscore(train['Attrition_rate'])

In [30]:
# train = train[train['zscore'].abs() <= 3].drop(['zscore'],axis=1)

In [31]:
types = {
    'type0':'M2',
    'type1':'M2',
    'type2':'L2',
    'type4':'L2',
    'type3':'L2'
}

In [32]:
all_data['COMPType'] = all_data['Compensation_and_Benefits'].map(types)

In [33]:
test_eid = test['Employee_ID'].copy()

In [34]:
# pvar = 'Hometown'

In [35]:
# le = LabelEncoder()
# all_data[pvar] = le.fit_transform(all_data[pvar])

In [36]:
all_data['Employee_ID'] = all_data['Employee_ID'].apply(lambda x: int(x.split('_')[1]))

In [37]:
cols = all_data.columns[all_data.dtypes == 'object']
for col in cols:
    all_data[col].interpolate(method='linear',inplace=True)

In [38]:
def age_bin(s):
    return s['Age']/10

def emp_bin(s):
    if s['Employee_ID'] <= 7200:
        return 'OLD'
    if s['Employee_ID'] <= 18000:
        return 'MID'
    else:
        return 'NEW'

def growth_bin(s):
    if s['growth_rate'] <= 33:
        return 'NOT Satisfy'
    if s['growth_rate'] <= 47:
        return 'Satisfy'
    else:
        return 'High Satisfy'


In [39]:
all_data['age_bins'] = all_data.apply(age_bin,axis=1)

all_data['emp_bins'] = all_data.apply(emp_bin,axis=1)

all_data['growth_bins'] = all_data.apply(growth_bin,axis=1)

In [40]:
cat_features = list(all_data.columns[all_data.dtypes == 'object'])

In [41]:
toap = (['Education_Level','Age','Travel_Rate','Post_Level','Work_Life_balance','Pay_Scale','Retiring_or_not'])

In [42]:
for f in toap:
    cat_features.append(f)

In [43]:
for i in range(1,8):
    name = 'VAR'+str(i)
    cat_features.append(name)

In [44]:
for cat in cat_features:
    g_var = cat
    mediann = dict(all_data.groupby(g_var)['Attrition_rate'].mean())
    minn = dict(all_data.groupby(g_var)['Attrition_rate'].min())
    maxx = dict(all_data.groupby(g_var)['Attrition_rate'].max())
    all_data[str(cat)+'_Avg'] = all_data[g_var].map(mediann)
    all_data[str(cat)+'_Min'] = all_data[g_var].map(minn)
    all_data[str(cat)+'_Max'] = all_data[g_var].map(maxx)

In [45]:
cols = all_data.columns
cols.delete(23)
for col in test.columns:
    try:
        all_data[col].replace([np.inf,-np.inf],np.nan,inplace=True)
        all_data[col].interpolate(method='linear',inplace=True)
    except:
        continue

In [46]:
train = all_data[~pd.isna(all_data['Attrition_rate'])].copy()
test = all_data[pd.isna(all_data['Attrition_rate'])].copy()

In [47]:
cols = train.columns[train.dtypes == 'object']
encoded = pd.DataFrame([])

X = train[cols]
y = train['Attrition_rate'].copy().values

for tr_in, val_in in KFold(shuffle = True).split(X, y):
    enc = TargetEncoder()
    enc.fit(X.iloc[tr_in], y[tr_in])
    encoded = encoded.append(enc.transform(X.iloc[val_in]))

enc = TargetEncoder()
train[cols] = enc.fit(X[cols],y)
test[cols] = enc.transform(test[cols])
train[cols] = encoded.sort_index()

"""
for col in cols:
  dummies = pd.get_dummies(train[col])
  train.drop([col],axis=1,inplace=True)
  train = pd.concat([train,dummies],axis=1)
  dummies = pd.get_dummies(test[col])
  test.drop([col],axis=1,inplace=True)
  test = pd.concat([test,dummies],axis=1)
"""
"""
for col in test.columns:
    if train[col].isnull().any():
        model = LGBMRegressor(**grid_lgbm)
        t1 = train[~pd.isna(train[col])].copy()
        t1.drop(['Attrition_rate'],axis=1,inplace=True)
        t2 = train[pd.isna(train[col])].copy()
        t2.drop(['Attrition_rate','Age'],axis=1,inplace=True)
        t3 = test[pd.isna(test[col])].copy()
        t3.drop(['Age'],axis=1,inplace=True)
        model.fit(t1.drop([col],axis=1),t1[col])
        train.loc[t2.index,col] = model.predict(t2)
        test.loc[t3.index,col] = model.predict(t3)
"""
"""
for cols in test.columns:
    train[cols].fillna(train[cols].mean(),inplace=True)
    test[cols].fillna(test[cols].mean(),inplace=True)
"""
pass

In [48]:
funcs = [np.multiply, np.add, np.subtract, np.divide]

for i, func in enumerate(funcs):
    train['Age_TOS'+str(i+1)] = func(train['Age'],train['Time_of_service'])
    test['Age_TOS'+str(i+1)] = func(test['Age'],test['Time_of_service'])

    train['AT_GR'+str(i+1)] = func(train['Age_TOS'+str(i+1)], train['growth_rate'])
    test['AT_GR'+str(i+1)] = func(test['Age_TOS'+str(i+1)], test['growth_rate'])

    train['TS_GR'+str(i+1)] = func(train['Time_of_service'], train['growth_rate'])
    test['TS_GR'+str(i+1)] = func(test['Time_of_service'], test['growth_rate'])

    train['PS_WLB'+str(i+1)] = func(train['Pay_Scale'], train['Work_Life_balance'])
    test['PS_WLB'+str(i+1)] = func(test['Pay_Scale'], test['Work_Life_balance'])

    train['Age_TSP'+str(i+1)] = func(train['Age'], train['Time_since_promotion'])
    test['Age_TSP'+str(i+1)] = func(test['Age'],test['Time_since_promotion'])

    train['Age_EMP'+str(i+1)] = func(train['Age'], train['Employee_ID'])
    test['Age_EMP'+str(i+1)] = func(test['Age'], test['Employee_ID'])

    train['TOS_EMP'+str(i+1)] = func(train['Time_of_service'],train['Employee_ID'])
    test['TOS_EMP'+str(i+1)] = func(test['Time_of_service'],test['Employee_ID'])

    train['ATOS_EMP'+str(i+1)] = func(train['Age_TOS'+str(i+1)], train['Employee_ID'])
    test['ATOS_EMP'+str(i+1)] = func(test['Age_TOS'+str(i+1)], test['Employee_ID'])


var_names = ['VAR'+str(i) for i in range(1,8)]
sums_tr = [] 
subs_tr = [] 
mul_tr = [] 
div_tr = []
sums_ts = []
subs_ts = []
mul_ts = []
div_ts = []
for row in train[var_names].itertuples():
    sums_tr.append(row[1]+row[2]+row[3]+row[4]+row[5]+row[6]+row[7])
    subs_tr.append(row[1]-row[2]-row[3]-row[4]-row[5]-row[6]-row[7])
    mul_tr.append(row[1]*row[2]*row[3]*row[4]*row[5]*row[6]*row[7])
    div_tr.append(row[1]/row[2]/row[3]/row[4]/row[5]/row[6]/row[7])
    
for row2 in test[var_names].itertuples():
    sums_ts.append(row[1]+row[2]+row[3]+row[4]+row[5]+row[6]+row[7])
    subs_ts.append(row[1]-row[2]-row[3]-row[4]-row[5]-row[6]-row[7])
    mul_ts.append(row[1]*row[2]*row[3]*row[4]*row[5]*row[6]*row[7])
    div_ts.append(row[1]/row[2]/row[3]/row[4]/row[5]/row[6]/row[7])
    
train['sum_VARS'] = sums_tr
train['sub_VARS'] = subs_tr
train['mul_VARS'] = mul_tr
train['div_VARS'] = div_tr
test['sum_VARS'] = sums_ts
test['sub_VARS'] = subs_ts
test['mul_VARS'] = mul_ts
test['div_VARS'] = div_ts

train['mulvars_gr'] = train['growth_rate'] * train['mul_VARS']
test['mulvars_gr'] = test['growth_rate'] * test['mul_VARS']

for col in test.columns:
    try:
        train[col].replace([np.inf,-np.inf],np.nan,inplace=True)
        test[col].replace([np.inf,-np.inf],np.nan,inplace=True)
        train[col].interpolate(method='linear',inplace=True)
        test[col].interpolate(method='linear',inplace=True)
    except:
        continue

In [49]:
funcs = [np.multiply, np.add, np.subtract, np.divide]
func_symbs = ['*','+','-','/']

for c1 in cat_features[15:]:
    for c2 in cat_features[15:]:
        for f,s in zip(funcs,func_symbs):
            train[str(c1)+str(s)+str(c2)] = f(train[c1],train[c2])
            test[str(c1)+str(s)+str(c2)] = f(test[c1],test[c2])

In [50]:
for col in test.drop(['Attrition_rate'],axis=1).columns:
    train[col].replace([np.inf,-np.inf],np.nan,inplace=True)
    test[col].replace([np.inf,-np.inf],np.nan,inplace=True)
    train[col].interpolate(method='linear',inplace=True)
    test[col].interpolate(method='linear',inplace=True)

In [51]:
kmeans = KMeans(n_clusters=12)

In [52]:
test['age_bins'].fillna(0,inplace=True)

In [53]:
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)

In [54]:
train['clusters'] = kmeans.fit_predict(train.drop(['Attrition_rate'],axis=1), train['Attrition_rate'])
test['clusters'] = kmeans.predict(test.drop(['Attrition_rate'],axis=1))

g_var = 'clusters'
mediann = dict(train.groupby(g_var)['Attrition_rate'].mean())
train[str(g_var)+'_Avg'] = train[g_var].map(mediann)
test[str(g_var)+'_Avg'] = test[g_var].map(mediann)

In [55]:
X = train.drop(['Attrition_rate'],axis=1)
y = train['Attrition_rate'].copy().values

In [56]:
test.drop(['Attrition_rate'],axis=1,inplace=True)

In [57]:
# gml = Ghalat_Machine_Learning()

# gml.GMLRegressor(X, y, test_Size = 0.4, metric = accuracy)

In [58]:
lr = LinearRegression()
lr.fit(X, y)
X['Preds'] = lr.predict(X)
test['Preds'] = lr.predict(test)

In [59]:
nb = LGBMRegressor()
nb.fit(X, y)
"""
fi = pd.DataFrame()
fi['A'] = X.columns.values
fi['B'] = lgbm.feature_importances_
"""
pass

In [60]:
fi = pd.DataFrame()
fi['Name'] = X.columns 
fi['importance'] = nb.feature_importances_
fi.sort_values('importance',ascending=False,inplace=True)
fi

,Name,importance
952,Preds,206
60,pos_tos_tsp_Avg,90
62,pos_tos_tsp_Max,89
27,pos_tos_tsp,69
61,pos_tos_tsp_Min,51
...,...,...
512,Retiring_or_not-Travel_Rate,0
514,Retiring_or_not*Post_Level,0
515,Retiring_or_not+Post_Level,0
516,Retiring_or_not-Post_Level,0


In [61]:
thresh = 10

In [62]:
cols_to_d = fi[fi['importance'] < thresh].loc[:,'Name']

In [63]:
X_BR = X.drop(cols_to_d,axis=1)
test_BR = test.drop(cols_to_d,axis=1)

In [64]:
thresh = 100
cols_to_d = fi[fi['importance'] < thresh].loc[:,'Name']

In [65]:
X_LR = X.drop(cols_to_d,axis=1)
test_LR = test.drop(cols_to_d,axis=1)

In [66]:
thresh = 10
cols_to_d = fi[fi['importance'] < thresh].loc[:,'Name']

In [67]:
X_L = X.drop(cols_to_d,axis=1)
test_L = test.drop(cols_to_d,axis=1)

In [68]:
thresh = 10
cols_to_d = fi[fi['importance'] < thresh].loc[:,'Name']

In [69]:
X_XGB = X.drop(cols_to_d,axis=1)
test_XGB = test.drop(cols_to_d,axis=1)

In [70]:
thresh = 50
cols_to_d = fi[fi['importance'] < thresh].loc[:,'Name']

In [71]:
X_lars = X.drop(cols_to_d,axis=1)
test_lars = test.drop(cols_to_d,axis=1)

In [72]:
thresh = 50
cols_to_d = fi[fi['importance'] < thresh].loc[:,'Name']

In [73]:
X_lgbm = X.drop(cols_to_d,axis=1)
test_lgbm = test.drop(cols_to_d,axis=1)

In [74]:
ss = pd.DataFrame()
ss['Employee_ID'] = test_eid
ss['Attrition_rate'] = np.nan

In [75]:
p1, p2, p3, p4 = 0.03, 0.00, 1.02, 0.0

In [76]:
def checkPerformance(model, X, y):
    scores = []

    for tr_in, val_in in KFold(shuffle=True).split(X, y):
        model.fit(X.iloc[tr_in], y[tr_in])
        preds = model.predict(X.iloc[val_in])
        scores.append(accuracy(y[val_in],preds))
    return np.mean(scores)

In [77]:
scores = []

for tr_in, val_in in KFold(shuffle=True).split(X_L, y):
    br = BayesianRidge()
    lr = Ridge()
    lasso = Lasso()
    lars = LassoLars()
    
    br.fit(X_BR.iloc[tr_in], y[tr_in])
    lr.fit(X_LR.iloc[tr_in], y[tr_in])
    lasso.fit(X_L.iloc[tr_in], y[tr_in])
    lars.fit(X_lars.iloc[tr_in], y[tr_in])
    
    preds = (br.predict(X_BR.iloc[val_in]) * p1 + lr.predict(X_LR.iloc[val_in]) * p2 + 
                        lasso.predict(X_L.iloc[val_in]) * p3 + lars.predict(X_lars.iloc[val_in]) * p4)
    scores.append(accuracy(y[val_in],preds))

In [78]:
scores

[81.16772946812493,
 82.50876489709293,
 81.85547448343688,
 81.15166440222403,
 81.1674679745278]

In [79]:
np.mean(scores)

81.5702202450813

In [80]:
# sr = StackingCVRegressor([BayesianRidge(), Ridge(), Lasso(), LassoLars()], BayesianRidge())

In [81]:
br = BayesianRidge()
lr = Ridge()
lasso = Lasso()
lars = LassoLars()

br.fit(X_BR, y)
lr.fit(X_LR, y)
lasso.fit(X_L, y)
lars.fit(X_lars, y)

pass

In [82]:
test_L.fillna(0,inplace=True)
test_LR.fillna(0,inplace=True)
test_BR.fillna(0,inplace=True)
test_lars.fillna(0,inplace=True)
test_XGB.fillna(0,inplace=True)
test_lgbm.fillna(0,inplace=True)

In [83]:
preds = (br.predict(test_BR) * p1 + lr.predict(test_LR) * p2 + lasso.predict(test_L) * p3 +
        lars.predict(test_lars) * p4 )

In [84]:
ss['Attrition_rate'] = preds

In [168]:
high = pd.read_csv('/kaggle/input/81636-he/81636.csv')['Attrition_rate']
high7 = pd.read_csv('/kaggle/input/81698-f/81698.csv')['Attrition_rate']
high6 = pd.read_csv('/kaggle/input/81692-he/81692.csv')['Attrition_rate']
high5 = pd.read_csv('/kaggle/input/81679-he/81679.csv')['Attrition_rate']
high4 = pd.read_csv('/kaggle/input/81645-he/81645.csv')['Attrition_rate']
high3 = pd.read_csv('/kaggle/input/81602-he/81603.csv')['Attrition_rate']
high2 = pd.read_csv('/kaggle/input/81597-he/81597.csv')['Attrition_rate']
high1 = pd.read_csv('/kaggle/input/81524-he/81524.csv')['Attrition_rate']
high0 = pd.read_csv('/kaggle/input/815701-he/815701.csv')['Attrition_rate']

b0 = pd.read_csv('/kaggle/input/81625-he/81625.csv')['Attrition_rate']
b1 = pd.read_csv('/kaggle/input/81616-he/81616.csv')['Attrition_rate']
b2 = pd.read_csv('/kaggle/input/81611-he/81611.csv')['Attrition_rate']
b3 = pd.read_csv('/kaggle/input/81607-he/81607.csv')['Attrition_rate']
b4 = pd.read_csv('/kaggle/input/81605-he/81605.csv')['Attrition_rate']
b5 = pd.read_csv('/kaggle/input/81602-he/81603.csv')['Attrition_rate']
b6 = pd.read_csv('/kaggle/input/816-he/816.csv')['Attrition_rate']
b7 = pd.read_csv('/kaggle/input/81599-he/81599.csv')['Attrition_rate']
b8 = pd.read_csv('/kaggle/input/81597-he/81597.csv')['Attrition_rate']
b9 = pd.read_csv('/kaggle/input/81586-he/81586.csv')['Attrition_rate']
b10 = pd.read_csv('/kaggle/input/815701-he/815701.csv')['Attrition_rate']
b11 = pd.read_csv('/kaggle/input/81566-he/81566.csv')['Attrition_rate']
b12 = pd.read_csv('/kaggle/input/81544-he/81544.csv')['Attrition_rate']
b13 = pd.read_csv('/kaggle/input/81528-he/81528.csv')['Attrition_rate']
b14 = pd.read_csv('/kaggle/input/81524-he/81524.csv')['Attrition_rate']
b15 = pd.read_csv('/kaggle/input/815-he/815.csv')['Attrition_rate']
b16 = pd.read_csv('/kaggle/input/81499-he/81499.csv')['Attrition_rate']
b17 = pd.read_csv('/kaggle/input/81497-he/submission(825).csv')['Attrition_rate']
b18 = pd.read_csv('/kaggle/input/81494-he/sexy_sub.csv')['Attrition_rate']
b19 = pd.read_csv('/kaggle/input/81486-he/81486.csv')['Attrition_rate']


pred = pd.read_csv('/kaggle/input/81544-he/81544.csv')['Attrition_rate']
low = pd.read_csv('/kaggle/input/81499-he/81499.csv')['Attrition_rate']
low1 = pd.read_csv('/kaggle/input/8134704-he/8134704.csv')['Attrition_rate']
low0 = pd.read_csv('/kaggle/input/81314-he/81314.csv')['Attrition_rate']

pred1 = br.predict(test_BR) 
pred2 = lr.predict(test_LR) 
pred3 = lasso.predict(test_L) 
pred4 = lars.predict(test_lars)

In [86]:
# p = 0.5 * low0 + 0.5 * low1

In [87]:
# h1 = (high+high3)/2

In [88]:
# ss['Attrition_rate'] = (b1+b2+b3+b4+b5+b6)/6

In [89]:
# ss['Attrition_rate'] = high + np.abs(high4-high3)**2

In [90]:
smexy = ss['Attrition_rate'] = (b0+b3) - b6

In [91]:
smexy2 = ss['Attrition_rate'] = (smexy+b7) - b6

In [169]:
bs = [high7,b0,b1,b2,b3,b4,b5,b6,b7,b8,b9,b10,b11,b12,b13,b14,b15,b16,b17,b18,b19]

In [230]:
"""
for j,bs2 in enumerate(bs):
    for k,bs3 in enumerate(bs):
        if ( j == k ):
            continue
        comb = (high7 + bs2) - (bs3)
        if comb.describe()['max'] <= 0.99 and comb.describe()['min'] > 0 and (0.197 <= comb.describe()['mean'] <= 0.198):
            print(0, ' ', j, ' ', k)
            print(comb.describe())
            print()
"""
pass

In [231]:
good_sub = high7

In [282]:
for j,bs2 in enumerate(bs):
    for k,bs3 in enumerate(bs):
        if (j == k):
            continue
        comb = 30*(good_sub + np.abs((good_sub+bs2+bs3)/3)**8) - 29*good_sub
        if comb.describe()['max'] <= 0.99 and comb.describe()['min'] > 0 and (0.197 <= comb.describe()['mean'] <= 0.1985):
            print(j, ' ', k)
            print(comb.describe())
            print()

1   2
count    3000.000000
mean        0.197969
std         0.044502
min         0.099800
25%         0.172607
50%         0.191356
75%         0.215001
max         0.985074
Name: Attrition_rate, dtype: float64

1   3
count    3000.000000
mean        0.197925
std         0.044048
min         0.099800
25%         0.172608
50%         0.191359
75%         0.215001
max         0.951062
Name: Attrition_rate, dtype: float64

1   4
count    3000.000000
mean        0.197942
std         0.044112
min         0.099800
25%         0.172608
50%         0.191361
75%         0.215005
max         0.951583
Name: Attrition_rate, dtype: float64

1   5
count    3000.000000
mean        0.198128
std         0.045188
min         0.099800
25%         0.172608
50%         0.191361
75%         0.215006
max         0.956527
Name: Attrition_rate, dtype: float64

1   6
count    3000.000000
mean        0.197927
std         0.044258
min         0.099800
25%         0.172596
50%         0.191332
75%         0.214990

3   10
count    3000.000000
mean        0.197956
std         0.043959
min         0.099800
25%         0.172611
50%         0.191357
75%         0.215011
max         0.908448
Name: Attrition_rate, dtype: float64

3   11
count    3000.000000
mean        0.197755
std         0.043381
min         0.020968
25%         0.172597
50%         0.191343
75%         0.214993
max         0.909638
Name: Attrition_rate, dtype: float64

3   12
count    3000.000000
mean        0.197755
std         0.043393
min         0.017146
25%         0.172596
50%         0.191342
75%         0.214993
max         0.909658
Name: Attrition_rate, dtype: float64

3   13
count    3000.000000
mean        0.197752
std         0.043335
min         0.018656
25%         0.172600
50%         0.191346
75%         0.214998
max         0.903217
Name: Attrition_rate, dtype: float64

3   14
count    3000.000000
mean        0.197740
std         0.043324
min         0.004829
25%         0.172599
50%         0.191349
75%         0.2

6   1
count    3000.000000
mean        0.197927
std         0.044258
min         0.099800
25%         0.172596
50%         0.191332
75%         0.214990
max         0.950877
Name: Attrition_rate, dtype: float64

6   2
count    3000.000000
mean        0.197885
std         0.043923
min         0.099800
25%         0.172609
50%         0.191329
75%         0.214992
max         0.949865
Name: Attrition_rate, dtype: float64

6   3
count    3000.000000
mean        0.197839
std         0.043549
min         0.099800
25%         0.172608
50%         0.191336
75%         0.214992
max         0.924812
Name: Attrition_rate, dtype: float64

6   4
count    3000.000000
mean        0.197851
std         0.043568
min         0.099800
25%         0.172609
50%         0.191363
75%         0.214995
max         0.925193
Name: Attrition_rate, dtype: float64

6   7
count    3000.000000
mean        0.197806
std         0.043681
min         0.003632
25%         0.172598
50%         0.191339
75%         0.215000

8   12
count    3000.000000
mean        0.197760
std         0.043340
min         0.023341
25%         0.172596
50%         0.191344
75%         0.214997
max         0.905554
Name: Attrition_rate, dtype: float64

8   13
count    3000.000000
mean        0.197757
std         0.043286
min         0.025403
25%         0.172600
50%         0.191348
75%         0.215002
max         0.899623
Name: Attrition_rate, dtype: float64

8   14
count    3000.000000
mean        0.197741
std         0.043280
min         0.006010
25%         0.172599
50%         0.191352
75%         0.215007
max         0.899692
Name: Attrition_rate, dtype: float64

8   15
count    3000.000000
mean        0.197744
std         0.043283
min         0.007674
25%         0.172602
50%         0.191356
75%         0.215012
max         0.900245
Name: Attrition_rate, dtype: float64

8   16
count    3000.000000
mean        0.197720
std         0.043140
min         0.005699
25%         0.172602
50%         0.191349
75%         0.2

11   1
count    3000.000000
mean        0.197788
std         0.043740
min         0.013933
25%         0.172596
50%         0.191338
75%         0.215000
max         0.930067
Name: Attrition_rate, dtype: float64

11   2
count    3000.000000
mean        0.197782
std         0.043650
min         0.020196
25%         0.172595
50%         0.191336
75%         0.215002
max         0.929271
Name: Attrition_rate, dtype: float64

11   3
count    3000.000000
mean        0.197755
std         0.043381
min         0.020968
25%         0.172597
50%         0.191343
75%         0.214993
max         0.909638
Name: Attrition_rate, dtype: float64

11   4
count    3000.000000
mean        0.197761
std         0.043393
min         0.023529
25%         0.172598
50%         0.191351
75%         0.214996
max         0.909935
Name: Attrition_rate, dtype: float64

11   5
count    3000.000000
mean        0.197808
std         0.043520
min         0.036403
25%         0.172598
50%         0.191351
75%         0.2

13   10
count    3000.000000
mean        0.197743
std         0.043132
min         0.027469
25%         0.172601
50%         0.191349
75%         0.215008
max         0.892051
Name: Attrition_rate, dtype: float64

13   11
count    3000.000000
mean        0.197732
std         0.043162
min         0.002830
25%         0.172600
50%         0.191360
75%         0.215001
max         0.892851
Name: Attrition_rate, dtype: float64

13   12
count    3000.000000
mean        0.197734
std         0.043168
min         0.002809
25%         0.172599
50%         0.191359
75%         0.215002
max         0.892864
Name: Attrition_rate, dtype: float64

13   14
count    3000.000000
mean        0.197726
std         0.043106
min         0.002776
25%         0.172603
50%         0.191368
75%         0.215012
max         0.888602
Name: Attrition_rate, dtype: float64

13   15
count    3000.000000
mean        0.197728
std         0.043109
min         0.002777
25%         0.172606
50%         0.191374
75%       

15   18
count    3000.000000
mean        0.197701
std         0.042964
min         0.002776
25%         0.172605
50%         0.191371
75%         0.215010
max         0.881696
Name: Attrition_rate, dtype: float64

15   19
count    3000.000000
mean        0.197705
std         0.042977
min         0.002776
25%         0.172607
50%         0.191375
75%         0.215018
max         0.883623
Name: Attrition_rate, dtype: float64

15   20
count    3000.000000
mean        0.197706
std         0.042977
min         0.002776
25%         0.172606
50%         0.191377
75%         0.215015
max         0.883553
Name: Attrition_rate, dtype: float64

16   1
count    3000.000000
mean        0.197745
std         0.043447
min         0.003731
25%         0.172601
50%         0.191341
75%         0.214988
max         0.909532
Name: Attrition_rate, dtype: float64

16   2
count    3000.000000
mean        0.197739
std         0.043382
min         0.004517
25%         0.172600
50%         0.191335
75%         

count    3000.000000
mean        0.197719
std         0.043166
min         0.004968
25%         0.172601
50%         0.191352
75%         0.214994
max         0.892543
Name: Attrition_rate, dtype: float64

18   5
count    3000.000000
mean        0.197734
std         0.043208
min         0.006904
25%         0.172601
50%         0.191353
75%         0.214994
max         0.894416
Name: Attrition_rate, dtype: float64

18   6
count    3000.000000
mean        0.197722
std         0.043170
min         0.003006
25%         0.172601
50%         0.191351
75%         0.214994
max         0.892276
Name: Attrition_rate, dtype: float64

18   7
count    3000.000000
mean        0.197715
std         0.043112
min         0.002776
25%         0.172601
50%         0.191347
75%         0.214998
max         0.889535
Name: Attrition_rate, dtype: float64

18   8
count    3000.000000
mean        0.197715
std         0.043119
min         0.005692
25%         0.172601
50%         0.191346
75%         0.214995
m

20   14
count    3000.000000
mean        0.197703
std         0.042974
min         0.002776
25%         0.172604
50%         0.191371
75%         0.215010
max         0.883228
Name: Attrition_rate, dtype: float64

20   15
count    3000.000000
mean        0.197706
std         0.042977
min         0.002776
25%         0.172606
50%         0.191377
75%         0.215015
max         0.883553
Name: Attrition_rate, dtype: float64

20   16
count    3000.000000
mean        0.197689
std         0.042882
min         0.002776
25%         0.172604
50%         0.191370
75%         0.214996
max         0.878723
Name: Attrition_rate, dtype: float64

20   17
count    3000.000000
mean        0.197688
std         0.042881
min         0.002776
25%         0.172604
50%         0.191369
75%         0.214995
max         0.878926
Name: Attrition_rate, dtype: float64

20   18
count    3000.000000
mean        0.197685
std         0.042869
min         0.002776
25%         0.172603
50%         0.191369
75%       

In [309]:
ss['Attrition_rate'] = 30*(good_sub + np.abs((good_sub+b4+b3)/3)**8) - 29*good_sub

In [310]:
ss.to_csv('submission.csv',index=False)

In [311]:
ss.describe()

,Attrition_rate
count,3000.000000
mean,0.197980
std,0.044165
min,0.099800
25%,0.172611
50%,0.191360
75%,0.215023
max,0.908738


In [312]:
"""
good ones

ss['Attrition_rate'] = (b1+b3) - b8
ss['Attrition_rate'] = (b1+b5) - b11
ss['Attrition_rate'] = (b0+b4) - b6
ss['Attrition_rate'] = (smexy+b2) - b6
ss['Attrition_rate'] = (b0+b3) - b6
ss['Attrition_rate'] = (smexy+b5) - b6
ss['Attrition_rate'] = (smexy+b7) - b6
ss['Attrition_rate'] = (smexy2+b7) - b6
ss['Attrition_rate'] = (high + b2) - b7
ss['Attrition_rate'] = (high + b8) - b7
ss['Attrition_rate'] = (high + b9) - b7
ss['Attrition_rate'] = (good_sub2 + b2) - b3
ss['Attrition_rate'] = (good_sub2 + b8) - b5
ss['Attrition_rate'] = (high4 + b8) - b9
ss['Attrition_rate'] = 50*(good_sub + np.abs(high4-b1)**2 - np.abs(high4-b1)**2.9) - 49*good_sub
ss['Attrition_rate'] = (high5 + b0) - high
ss['Attrition_rate'] = (high6 + b5) - b7
ss['Attrition_rate'] = 30*(good_sub + np.abs((high7+b15)/2)**7) - 29*good_sub


"""
pass